# 3.3 (補充) LLM Information Extraction

HuggingFace 模型來實作「PTT以巴衝突議題」的情緒分析。

# 連接資料夾 & 環境 & Token 設定

## 連接雲端資料夾

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/2024SMA/week13') #切換該目錄
os.listdir() #確認目錄內容

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['LLM_information_extraction.ipynb', 'requirements.txt', 'dict', 'data']

##  套件安裝

In [ ]:
!pip install jieba

!pip install sentence-transformers
!pip install transformers
!pip install sentencepiece

# 用於 transformer 加速
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install langchain-chroma
!pip install pypdf
!pip install langchain
!pip install huggingface_hub
!pip install grandalf
!pip install pandas
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
import os
from getpass import getpass

import pandas as pd
import ast
import json
import re
import jieba

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import BitsAndBytesConfig # huggingface 量化

from langchain_community.llms import HuggingFaceEndpoint

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

## 輸入 HuggingFace Token
- get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


In [ ]:
quant_config = BitsAndBytesConfig(load_in_4bit=True)

print(quant_config.is_quantizable())
print(quant_config.quantization_method())

True
fp4


In [ ]:
model_id = "google/gemma-1.1-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HUGGINGFACEHUB_API_TOKEN)
# device_map="auto" 使用 GPU 加速
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quant_config, token=HUGGINGFACEHUB_API_TOKEN)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
# repo_id: HuggingFace 上的模型名稱，本範例使用 gemma 7b instruct 版本
    # huggingface: https://huggingface.co/google/gemma-1.1-7b-it
# temperature: 模型回答的活潑程度，0~1 越大每次回答的多樣性越高

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-1.1-7b-it", temperature=1.0,model_kwargs={'token':HUGGINGFACEHUB_API_TOKEN, 'use_cache':False}
)
llm.invoke("你好，自然語言處理有什麼相關應用？")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


'\n\n**自然語言處理的相關應用：**\n\n**1. 資訊摘要**\n\n* 自動化新聞報導、網頁和文件摘要。\n* 協助使用者快速了解文章內容。\n\n\n**2. 情緒分析**\n\n* 分析文本的情緒並評估其積極或消極程度。\n* 用於廣告投放、客戶回饋和社會媒體內容管理。\n\n\n**3. 情感偵測**\n\n* 利用自然語言處理技術來偵測人類的情緒和情緒變化。\n* 用於人機介面和客戶服務。\n\n\n**4. 語法檢查**\n\n* 自動檢查文檔的語法錯誤。\n* 用於線上撰寫工具和文字處理軟體。\n\n\n**5. 詞彙典範建立**\n\n* 建立和管理詞彙典範。\n* 用於機器翻譯和自然語言理解。\n\n\n**6. 文字分類**\n\n* 將文本分類到特定的類別。\n* 用於電子郵件過濾和新聞報導整理。\n\n\n**7. 語義分析**\n\n* 分析文本的語義結構和含義。\n* 用於機器翻譯和自然語言理解。\n\n\n**8. 自然語言生成**\n\n* 基於給定的一組詞彙或概念，自動生成完整的句子或段落。\n* 用於聊天機器人和語音助理。\n\n\n**9. 人機介面**\n\n* 開發人機介面，讓人類和機器更有效地進行互動。\n\n\n**10. 社交媒體分析**\n\n* 分析社交媒體數據以了解使用者行為和趨勢。\n\n**自然語言處理的未來應用：**\n\n* 更進階的語義理解和自然語言生成。\n* 人機合作和自動化。\n* 基於自然語言處理的個人化體驗。'

# 資料集實作 Information Extraction (Tagging)
- https://python.langchain.com/docs/use_cases/tagging/

- 使用ptt政黑版、八卦版，2022/01/01~2024/04/16期間，關鍵字為「以巴衝突」、「以巴戰爭」、「以色列」、「巴勒斯坦」 的文章



## 3.3.1 前處理

In [ ]:
df = pd.read_csv("./data/sa_project1.csv")
df["artDate"] = pd.to_datetime(df["artDate"])
df['artDate'] = df['artDate'].dt.date

clear_df = df.copy()

#去除本次不會用到的欄位
drop_cols = ['artPoster', 'artCatagory', 'artComment', 'e_ip', 'insertedDate', 'dataSource']
clear_df.drop(drop_cols, axis = 1, inplace = True)

#去除文章內容為空值的筆數
clear_df.dropna(subset = ['artContent'], axis=0, how='any', inplace=True)

#新增['sentence']欄位，用'。'取代'\n\n'，並移除'\n'
clear_df['sentence'] = clear_df['artContent'].str.replace(r'\n\n','。', regex=True)
clear_df['sentence'] = clear_df['sentence'].str.replace(r'\n','', regex=True)

#移除內文中的網址
clear_df['sentence'] = clear_df['sentence'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

clear_df['sentence'] = clear_df['sentence'].str.split("[,，。！!？?]{1,}")

sent_df = clear_df.explode('sentence').reset_index(drop=True)
# 把長度小於1的sentence刪掉
sent_df = sent_df[sent_df["sentence"].str.len() > 1]

sampled_df = sent_df.sample(n=100, random_state=222)

sampled_df

,artUrl,artTitle,artDate,artContent,sentence
370,https://www.ptt.cc/bbs/Gossiping/M.1696670440....,[問卦]以巴戰爭發生核戰會影響到台灣嗎？,2023-10-07,至今為止還沒有發生過兩邊互丟核彈的戰爭\n有沒有這次戰爭如果不爽\n兩邊互丟 核彈\n丟到全...,至今為止還沒有發生過兩邊互丟核彈的戰爭有沒有這次戰爭如果不爽兩邊互丟 核彈丟到全部死光光會影...
9356,https://www.ptt.cc/bbs/Gossiping/M.1697452078....,[新聞]我能回台灣「以色列朋友沒有」 台人憂心,2023-10-16,媒體來源：TVBS新聞網\n\n記者署名：洪浚緯\n\n完整新聞標題：我能回台灣「以色列朋友...,但是我又我
24965,https://www.ptt.cc/bbs/HatePolitics/M.16975284...,[新聞]徐旭東：國際訂單回流ECFA以巴衝突不影,2023-10-17,1.新聞網址︰\nhttps://money.udn.com/money/story/561...,「沒有影響」
25892,https://www.ptt.cc/bbs/HatePolitics/M.16979920...,[新聞]半島電視台：解放軍不少人認為台海作戰沒,2023-10-23,1.新聞網址︰\nhttps://newtalk.tw/news/view/2023-10-...,不若則能避之
13146,https://www.ptt.cc/bbs/Gossiping/M.1698056570....,[新聞]哈馬斯創始人之子：恐怖組織不在乎巴人,2023-10-23,1.媒體來源:\n\n大紀元\n\n2.記者署名:\n\n大紀元記者李言綜合報導\n\n3....,【大紀元2023年10月22日訊】（大紀元記者李言綜合報導）以巴衝突持續之際
...,...,...,...,...,...
7059,https://www.ptt.cc/bbs/Gossiping/M.1697297875....,[新聞]「我是巴勒斯坦人，我哭泣。」施明德：,2023-10-14,https://udn.com/news/story/6656/7504348\n「我是巴勒...,並表示「現在是表態的時刻」
27959,https://www.ptt.cc/bbs/HatePolitics/M.17048186...,[討論]選前淺談,2024-01-10,"2024年1月9號,選前倒數4天,2024年大選神探v.s醫師選擇,各陣營最後衝刺中,大勢已...",加拿大/俄羅斯/印度/沙烏地阿拉伯/以色列/伊朗/阿聯/土耳其/韓國/朝鮮/南非/埃及/
6471,https://www.ptt.cc/bbs/Gossiping/M.1697253410....,[新聞]才炸死路透記者！BBC採訪團隊遭以色列警,2023-10-14,1.媒體來源:\n聯合新聞網\n\n2.記者署名:\n徐榆涵\n\n3.完整新聞標題:\n才...,儘管車上貼有明顯的媒體標誌
10207,https://www.ptt.cc/bbs/Gossiping/M.1697516697....,[新聞]對付哈馬斯以軍有不怕火箭和地雷的利器,2023-10-17,對付哈馬斯 以軍有不怕火箭和地雷的利器\nhttps://i.epochtimes.com/...,可用於更危險的環境中


## 3.3.2 判斷情緒跟立場

In [ ]:
class IsraelPalestineStance(BaseModel):
    情緒: str = Field(..., description="判斷整個句子總體的情緒。", enum=["正向", "中性", "負向"])
    涉及方: str = Field(
        ...,
        description="列出所有句子中提及的涉及方並以逗號','分隔。",
    )
    立場種類: str = Field(
        ...,
        description="判斷句子中提到的立場是屬於什麼種類的。",
        enum=["支持以色列", "支持巴勒斯坦", "批評以色列", "批評巴勒斯坦", "支持哈馬斯", "批評哈馬斯", "支持聯合國", "批評聯合國", "支持美國", "批評美國", "中立", "其他", "無"],
    )

stance_parser = JsonOutputParser(pydantic_object=IsraelPalestineStance)

# 透過 get_format_instructions() 可以看到如何告訴 LLM 生成我們想要的格式的一些指令
stance_parser.get_format_instructions()


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"\\u60c5\\u7dd2": {"title": "\\u60c5\\u7dd2", "description": "\\u5224\\u65b7\\u6574\\u500b\\u53e5\\u5b50\\u7e3d\\u9ad4\\u7684\\u60c5\\u7dd2\\u3002", "enum": ["\\u6b63\\u5411", "\\u4e2d\\u6027", "\\u8ca0\\u5411"], "type": "string"}, "\\u6d89\\u53ca\\u65b9": {"title": "\\u6d89\\u53ca\\u65b9", "description": "\\u5217\\u51fa\\u6240\\u6709\\u53e5\\u5b50\\u4e2d\\u63d0\\u53ca\\u7684\\u6d89\\u53ca\\u65b9\\u4e26\\u4ee5\\u9017\\u865f\',\'\\u5206\\u9694\\u3002", "type": "string"}, "\\u7acb\\u5834\\u7a2e\\u985e": {"title": "\\u7acb

### 定義模板

In [ ]:
# Set up a parser + inject instructions into the prompt template.
stance_parser = JsonOutputParser(pydantic_object=IsraelPalestineStance)\

stance_template = """<start_of_turn>user
你是一位時事評論家，你將會看網路上對於以巴戰爭的評論，請抓取出評論中：
1. 此評論的情緒，正向或負向或中性
2. 有討論到哪些涉及方，例如：以色列、巴勒斯坦、哈馬斯、聯合國、美國
3. 立場的種類，例如：支持以色列、支持巴勒斯坦、批評以色列、批評巴勒斯坦、支持哈馬斯、批評哈馬斯、支持聯合國、批評聯合國、支持美國、批評美國、中立、其他、無

以下為一些範例：
```
範例1
句子：我認為以色列在這場衝突中有其正當性
抓取結果：{{
"情緒":"正向",
"涉及方":"以色列",
"立場種類":"支持以色列"
}}

範例2
句子：這次衝突再次證明了哈馬斯的暴行
抓取結果：{{
"情緒":"負向",
"涉及方":"哈馬斯",
"立場種類":"批評哈馬斯"
}}

範例3
句子：巴勒斯坦人民一直在為自己的權利而奮鬥
抓取結果：{{
"情緒":"正向",
"涉及方":"巴勒斯坦",
"立場種類":"支持巴勒斯坦"
}}

{format_instructions}
請使用繁體中文回答。
除了 JSON 內容外，不要回答任何其他內容

現在請幫我抓取以下句子的資訊：
句子：{query}
抓取結果：<end_of_turn>
<start_of_turn>model
"""

stance_prompt = PromptTemplate(
  template=stance_template,
  # template="Answer the user query.\n{format_instructions}\nDon't output any other explanation\n{query}\n",
  input_variables=["query"],
  # 將 format_instructions 也輸入到 prompt templete 中
  partial_variables={"format_instructions": stance_parser.get_format_instructions()},
)

chain = stance_prompt | llm
chain.get_graph().print_ascii()

       +-------------+         
       | PromptInput |         
       +-------------+         
              *                
              *                
              *                
      +----------------+       
      | PromptTemplate |       
      +----------------+       
              *                
              *                
              *                
   +---------------------+     
   | HuggingFaceEndpoint |     
   +---------------------+     
              *                
              *                
              *                
+---------------------------+  
| HuggingFaceEndpointOutput |  
+---------------------------+  


### 手動建立 parse function

In [ ]:
# 從 json 字串中抓取答案
def extract_answer_from_string(string_):
    string_ = string_.replace("\n", "")
    string_ = string_.replace("json", "")
    string_ = string_.replace("    ", "")
    string_ = string_.replace("```", "")
    try:
        # 可以成功轉換
        dict_result = json.loads(string_)
    except:
        print(string_)
        return string_
    return dict_result

# 從 dict 抓取情緒
def get_sentiment(row):
    sentiment = None
    if isinstance(row, dict):
        if ("情緒" in row.keys()):
            sentiment = row['情緒']
    return sentiment

### 舉例判斷看看，情緒、涉及方、立場種類都算準確

In [ ]:
llm_result = extract_answer_from_string(chain.invoke({"query": "巴勒斯坦人民遭到襲擊致死，好恐怖"}))
print(llm_result)
get_sentiment(llm_result)

{'情緒': '負向', '涉及方': '巴勒斯坦人民', '立場種類': '批評以色列'}


'負向'

### 把資料放進來測試

In [ ]:
sampled_df['LLM'] = sampled_df.apply(lambda x: extract_answer_from_string(chain.invoke({"query": x['sentence']})), axis=1)
sampled_df

The provided text does not include any news headlines, so I am unable to extract the requested data from the given context.
The provided context does not include the actual sentence or any information regarding the sentiment, involved parties, or stance taken. Therefore, I am unable to extract the requested data from the given context.
無法抓取資訊，因為該句子並非一個客觀的評量，也並非所有世界的人都在支持以色列。
The provided text is not included in the given context, so I am unable to extract the requested information.
The provided sentence is not included in the given text, so I am unable to extract the requested data from the context.
"情緒":"負向","涉及方":"以色列","立場種類":"批評以色列"
The provided text does not include any sentence regarding Morgan and a program, so I am unable to extract the requested data from the given context.
The provided sentence is not included in the given context, so I am unable to extract the requested information.
The provided text does not include the actual sentence or its sentiment, so I am unable to ext

,artUrl,artTitle,artDate,artContent,sentence,LLM
370,https://www.ptt.cc/bbs/Gossiping/M.1696670440....,[問卦]以巴戰爭發生核戰會影響到台灣嗎？,2023-10-07,至今為止還沒有發生過兩邊互丟核彈的戰爭\n有沒有這次戰爭如果不爽\n兩邊互丟 核彈\n丟到全...,至今為止還沒有發生過兩邊互丟核彈的戰爭有沒有這次戰爭如果不爽兩邊互丟 核彈丟到全部死光光會影...,"{'情緒': '中性', '涉及方': ['以色列', '巴勒斯坦', '核彈'], '立場..."
9356,https://www.ptt.cc/bbs/Gossiping/M.1697452078....,[新聞]我能回台灣「以色列朋友沒有」 台人憂心,2023-10-16,媒體來源：TVBS新聞網\n\n記者署名：洪浚緯\n\n完整新聞標題：我能回台灣「以色列朋友...,但是我又我,"{'情緒': '中性', '涉及方': '我', '立場種類': '中立'}"
24965,https://www.ptt.cc/bbs/HatePolitics/M.16975284...,[新聞]徐旭東：國際訂單回流ECFA以巴衝突不影,2023-10-17,1.新聞網址︰\nhttps://money.udn.com/money/story/561...,「沒有影響」,"{'情緒': '中性', '涉及方': [], '立場種類': '中立'}"
25892,https://www.ptt.cc/bbs/HatePolitics/M.16979920...,[新聞]半島電視台：解放軍不少人認為台海作戰沒,2023-10-23,1.新聞網址︰\nhttps://newtalk.tw/news/view/2023-10-...,不若則能避之,"{'情緒': '中性', '涉及方': '以色列、巴勒斯坦、聯合國', '立場種類': '中立'}"
13146,https://www.ptt.cc/bbs/Gossiping/M.1698056570....,[新聞]哈馬斯創始人之子：恐怖組織不在乎巴人,2023-10-23,1.媒體來源:\n\n大紀元\n\n2.記者署名:\n\n大紀元記者李言綜合報導\n\n3....,【大紀元2023年10月22日訊】（大紀元記者李言綜合報導）以巴衝突持續之際,"{'情緒': '中性', '涉及方': '以色列、巴勒斯坦', '立場種類': '中立'}"
...,...,...,...,...,...,...
7059,https://www.ptt.cc/bbs/Gossiping/M.1697297875....,[新聞]「我是巴勒斯坦人，我哭泣。」施明德：,2023-10-14,https://udn.com/news/story/6656/7504348\n「我是巴勒...,並表示「現在是表態的時刻」,"{'情緒': '中性', '涉及方': '不適用', '立場種類': '不適用'}"
27959,https://www.ptt.cc/bbs/HatePolitics/M.17048186...,[討論]選前淺談,2024-01-10,"2024年1月9號,選前倒數4天,2024年大選神探v.s醫師選擇,各陣營最後衝刺中,大勢已...",加拿大/俄羅斯/印度/沙烏地阿拉伯/以色列/伊朗/阿聯/土耳其/韓國/朝鮮/南非/埃及/,"{'情緒': '中性', '涉及方': '加拿大、俄羅斯、印度、沙烏地阿拉伯、以色列、伊朗、..."
6471,https://www.ptt.cc/bbs/Gossiping/M.1697253410....,[新聞]才炸死路透記者！BBC採訪團隊遭以色列警,2023-10-14,1.媒體來源:\n聯合新聞網\n\n2.記者署名:\n徐榆涵\n\n3.完整新聞標題:\n才...,儘管車上貼有明顯的媒體標誌,"""情緒"":""負向"",""涉及方"":""以色列"",""立場種類"":""批評以色列"""
10207,https://www.ptt.cc/bbs/Gossiping/M.1697516697....,[新聞]對付哈馬斯以軍有不怕火箭和地雷的利器,2023-10-17,對付哈馬斯 以軍有不怕火箭和地雷的利器\nhttps://i.epochtimes.com/...,可用於更危險的環境中,"{'情緒': '正向', '涉及方': '聯合國', '立場種類': '支持聯合國'}"


當前提供的判斷情緒和立場的結論如下。

根據所提供的句子，我將對每個案例進行簡要分析，並判斷其情緒和立場分類是否準確：
*   Case 1
  *  句子: 至今為止還沒有發生過兩邊互丟核彈的戰爭有沒有這次戰爭如果不爽兩邊互丟核彈丟到全部死光光會...
  *  判斷: {'情緒': '中性', '涉及方': ['以色列', '巴勒斯坦', '核彈'], '立場種類': '中立'}
  *  分析: 情緒為中性合理，因為該句子只是陳述事實並提出假設，沒有明顯的情緒傾向。涉及方為以色列、巴勒斯坦和核彈也準確。立場種類為中立適當。

*   Case 2
  *  句子: 但是我又
  *  判斷: {'情緒': '中性', '涉及方': '我', '立場種類': '中立'}
  *  分析: 情緒為中性合理，因為該句子不完整且沒有明確的情緒表達。涉及方為“我”也準確。立場種類為中立適當。

*   Case 5
  *  句子: 【大紀元2023年10月22日訊】（大紀元記者李言綜合報導）以巴衝突持續之際
  *  判斷: {'情緒': '中性', '涉及方': '以色列、巴勒斯坦', '立場種類': '中立'}
  *  分析: 情緒為中性合理，因為該句子是新聞報導的一部分，通常是中立的。涉及方為以色列、巴勒斯坦也準確。立場種類為中立適當。


**綜合上述分析，這些判斷總體上是準確的。少數情況下可能需要更多上下文來進行更準確的判斷。**














### 判斷情緒

In [ ]:
sampled_df['sentiment'] = sampled_df.apply(lambda x: get_sentiment(x['LLM']), axis=1)
sampled_df.head()

,artUrl,artTitle,artDate,artContent,sentence,LLM,sentiment
370,https://www.ptt.cc/bbs/Gossiping/M.1696670440....,[問卦]以巴戰爭發生核戰會影響到台灣嗎？,2023-10-07,至今為止還沒有發生過兩邊互丟核彈的戰爭\n有沒有這次戰爭如果不爽\n兩邊互丟 核彈\n丟到全...,至今為止還沒有發生過兩邊互丟核彈的戰爭有沒有這次戰爭如果不爽兩邊互丟 核彈丟到全部死光光會影...,"{'情緒': '中性', '涉及方': ['以色列', '巴勒斯坦', '核彈'], '立場...",中性
9356,https://www.ptt.cc/bbs/Gossiping/M.1697452078....,[新聞]我能回台灣「以色列朋友沒有」 台人憂心,2023-10-16,媒體來源：TVBS新聞網\n\n記者署名：洪浚緯\n\n完整新聞標題：我能回台灣「以色列朋友...,但是我又我,"{'情緒': '中性', '涉及方': '我', '立場種類': '中立'}",中性
24965,https://www.ptt.cc/bbs/HatePolitics/M.16975284...,[新聞]徐旭東：國際訂單回流ECFA以巴衝突不影,2023-10-17,1.新聞網址︰\nhttps://money.udn.com/money/story/561...,「沒有影響」,"{'情緒': '中性', '涉及方': [], '立場種類': '中立'}",中性
25892,https://www.ptt.cc/bbs/HatePolitics/M.16979920...,[新聞]半島電視台：解放軍不少人認為台海作戰沒,2023-10-23,1.新聞網址︰\nhttps://newtalk.tw/news/view/2023-10-...,不若則能避之,"{'情緒': '中性', '涉及方': '以色列、巴勒斯坦、聯合國', '立場種類': '中立'}",中性
13146,https://www.ptt.cc/bbs/Gossiping/M.1698056570....,[新聞]哈馬斯創始人之子：恐怖組織不在乎巴人,2023-10-23,1.媒體來源:\n\n大紀元\n\n2.記者署名:\n\n大紀元記者李言綜合報導\n\n3....,【大紀元2023年10月22日訊】（大紀元記者李言綜合報導）以巴衝突持續之際,"{'情緒': '中性', '涉及方': '以色列、巴勒斯坦', '立場種類': '中立'}",中性


In [ ]:
sampled_df['sentiment'].value_counts()

sentiment
中性    40
負向    33
正向    15
無      1
Name: count, dtype: int64

**總體情緒中判斷為中向、負向者偏多。**